In [ ]:
# Jupyter seems to set the working directory to the parent of folder of the notebook so go up two levels
# this is necessary to have access to the axial_rot_analysis package

import os
os.chdir('../..')
import distutils.util
import quaternion as q
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from true_vs_apparent.common.database import create_db, BiplaneViconSubject
from true_vs_apparent.common.json_utils import get_params
from true_vs_apparent.common.analysis_utils import get_trajs, analyze_up_down, prepare_db


# initialize
config_dir = Path(r'C:\Users\klevis\Desktop\Repositories\axialAnalysis\config')
params = get_params(config_dir / 'parameters.json')

# ready db
db = create_db(params.biplane_vicon_db_dir, BiplaneViconSubject)
if params.excluded_trials:
    db = db[~db['Trial_Name'].str.contains('|'.join(params.excluded_trials))]
db = db.loc[[params.trial_name]]

# relevant parameters
use_ac = bool(distutils.util.strtobool(params.use_ac))

# compute min and max ht elevation for each subject
db['ht'], db['gh'], db['st'] = zip(*db['Trial'].apply(get_trajs, args=[db.attrs['dt'], params.torso_def, use_ac]))
db['up_down_analysis'] = db['ht'].apply(analyze_up_down)
prepare_db(db, params.torso_def, use_ac, params.dtheta_fine, params.dtheta_coarse, [params.min_elev, params.max_elev],
           should_fill=False, should_clean=False)
db_row = db.loc[params.trial_name]

In [ ]:
def create_axes():
    return [
        go.Scatter3d(x=[0, 1], y=[0, 0], z=[0, 0], mode='lines', line=dict(color='red', width=2), hoverinfo='none',
                     showlegend=False),
        go.Cone(x=[1], y=[0], z=[0], u=[0.25], v=[0], w=[0], colorscale=[[0, 'red'], [1, 'red']], showscale=False,
                anchor='tip', showlegend=False),
        go.Scatter3d(x=[0, 0], y=[0, 1], z=[0, 0], mode='lines', line=dict(color='green', width=2), hoverinfo='none',
                     showlegend=False),
        go.Cone(x=[0], y=[1], z=[0], u=[0], v=[0.25], w=[0], colorscale=[[0, 'green'], [1, 'green']], showscale=False,
                anchor='tip', showlegend=False),
        go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, 1], mode='lines', line=dict(color='blue', width=2), hoverinfo='none',
                     showlegend=False),
        go.Cone(x=[0], y=[0], z=[1], u=[0], v=[0], w=[0.25], colorscale=[[0, 'blue'], [1, 'blue']], showscale=False,
                anchor='tip', showlegend=False)]


In [ ]:
def create_sphere_phadke(r=1, num_divisions=11):
    # this is a generic globe with the poles pointing at +-x, the latitude and longitude below (although they are
    # related to) are not the same as elevation and plane of elevation
    # r should be 1, but for a publication figure I set it to slightly larger than 1 to enhance the color contrast

    # u is longitude degree
    # v is latitude degree
    u_lin = np.linspace(0, np.pi, num_divisions)
    v_lin = np.linspace(0, np.pi, num_divisions)
    broadcast_u = np.ones_like(u_lin)
    u, v = np.meshgrid(u_lin, v_lin)
    u = u.flatten()
    v = v.flatten()
    z = np.sin(u) * np.sin(v) * r
    x = np.cos(v) * r
    y = np.cos(u)*np.sin(v) * r

    lat_long_c = ['white'] * num_divisions
    lat_long_c[5] = 'grey'
    latitudes = [go.Scatter3d(x=broadcast_u * np.cos(lat) * r, y=np.cos(u_lin) * np.sin(lat) * r,
                              z=np.sin(u_lin) * np.sin(lat) * r, mode='lines', line=dict(color=lat_long_c[idx], width=2),
                              hoverinfo='none', showlegend=False) for idx, lat in enumerate(v_lin)]
    longitudes = [go.Scatter3d(x=np.cos(v_lin) * r, y=np.cos(long)*np.sin(v_lin) * r, z=np.sin(long)*np.sin(v_lin) * r,
                               mode='lines', line=dict(color=lat_long_c[idx], width=2), hoverinfo='none',
                               showlegend=False) for idx, long in enumerate(u_lin)]
    globe = go.Mesh3d(x=x, y=y, z=z, alphahull=0, opacity=0.2, color='cyan', hoverinfo='none', showlegend=False)
    return latitudes + longitudes + [globe]

In [ ]:
def create_sphere_isb(r=1, num_divisions=11):
    u_lin = np.linspace(-np.pi/2, np.pi/2, num_divisions)
    v_lin = np.linspace(0, np.pi, num_divisions)
    broadcast_u = np.ones_like(u_lin)
    u,v = np.meshgrid(u_lin, v_lin)
    u = u.flatten()
    v = v.flatten()
    x = np.sin(u) * np.sin(v) * r
    y = np.cos(v) * r
    z = np.cos(u) * np.sin(v) * r

    lat_long_c = ['white'] * num_divisions
    lat_long_c[5] = 'grey'
    latitudes = [go.Scatter3d(x=np.sin(u_lin)*np.sin(lat) * r, y=broadcast_u * np.cos(lat) * r,
                              z=np.cos(u_lin) * np.sin(lat) * r, mode='lines', line=dict(color=lat_long_c[idx], width=2),
                              hoverinfo='none', showlegend=False) for idx, lat in enumerate(v_lin)]
    longitudes = [go.Scatter3d(x=np.sin(long) * np.sin(v_lin) * r, y=np.cos(v_lin) * r, z=np.cos(long)*np.sin(v_lin) * r,
                               mode='lines', line=dict(color=lat_long_c[idx], width=2), hoverinfo='none',
                               showlegend=False) for idx, long in enumerate(u_lin)]
    globe = go.Mesh3d(x=x, y=y, z=z, alphahull=0, opacity=0.2, color='cyan', hoverinfo='none', showlegend=False)
    return latitudes + longitudes + [globe]

In [ ]:
def decomp_line_phadke(num_div=11):
    # the xy'z'' decomposition path from the start to end point is: 1) maintain elevation and go to zero PoE 2) raise arm
    # to final elevation angle 3) go to final PoE
    phadke_poe_traj = np.concatenate((np.linspace(poe_rec[0], 0, num_div), np.zeros((num_div,)),
                                      np.linspace(0, poe_rec[-1], num_div)))
    phadke_elev_traj = np.concatenate((np.ones((11,))*elev_rec[0], np.linspace(elev_rec[0], elev_rec[-1], num_div),
                                       np.ones((11,))*elev_rec[-1]))
    x_unit = np.array([1, 0, 0])
    z_unit = np.array([0, 0, 1])

    # easiest way of computing the path of the humeral axis (not to involve too much trig thinking), is to compute the
    # entire rotation matrix and extract just the y-axis
    phadke_traj_quat = [q.from_rotation_vector(x_unit * elev) * q.from_rotation_vector(z_unit * poe) for (elev, poe)
                        in zip(phadke_elev_traj, phadke_poe_traj)]
    phadke_traj_rotm = q.as_rotation_matrix(np.stack(phadke_traj_quat, axis=0))
    phadke_hum_axis = -phadke_traj_rotm[:, :, 1]
    phadke_x_traj = phadke_hum_axis[:, 0]
    phadke_y_traj = phadke_hum_axis[:, 1]
    phadke_z_traj = phadke_hum_axis[:, 2]
    return go.Scatter3d(name="xz'y''", x=phadke_x_traj, y=phadke_y_traj, z=phadke_z_traj, mode='lines',
                               line=dict(color='brown', width=6))

In [ ]:
def decomp_line_isb(num_div=11):
    # the yx'y'' decomposition path from the start to end point is: 1) from starting point go back to zero
    # 2) go to final orientation. There is the hidden caveat that since establishing the PoE counts as axial rotation
    # this must be included in the final computation
    isb_poe_traj = np.concatenate((np.ones((num_div,))*poe_rec[0], np.zeros((1,)), np.ones((num_div,))*poe_rec[-1]))
    isb_elev_traj = np.concatenate((np.linspace(elev_rec[0], 0, num_div), np.zeros((1,)), np.linspace(0, elev_rec[-1], num_div)))
    x_unit = np.array([1, 0, 0])
    y_unit = np.array([0, 1, 0])

    # easiest way of computing the path of the humeral axis (not to involve too much trig thinking), is to compute the
    # entire rotation matrix and extract just the y-axis
    isb_traj_quat = [q.from_rotation_vector(y_unit * poe) * q.from_rotation_vector(x_unit * elev) for (poe, elev)
                        in zip(isb_poe_traj, isb_elev_traj)]
    isb_traj_rotm = q.as_rotation_matrix(np.stack(isb_traj_quat, axis=0))
    isb_hum_axis = -isb_traj_rotm[:, :, 1]
    isb_x_traj = isb_hum_axis[:, 0]
    isb_y_traj = isb_hum_axis[:, 1]
    isb_z_traj = isb_hum_axis[:, 2]
    return go.Scatter3d(name="yx'y''", x=isb_x_traj, y=isb_y_traj, z=isb_z_traj, mode='lines',
                        line=dict(color='brown', width=6))

In [ ]:
def poe_elev_isb():
    poe_rec = db_row[params.trajectory].euler.yxy_intrinsic[start_idx:end_idx, 0]
    elev_rec = db_row[params.trajectory].euler.yxy_intrinsic[start_idx:end_idx, 1]
    return poe_rec, elev_rec

In [ ]:
def poe_elev_phadke():
    poe_rec = db_row[params.trajectory].euler.xzy_intrinsic[start_idx:end_idx, 1]
    elev_rec = db_row[params.trajectory].euler.xzy_intrinsic[start_idx:end_idx, 0]
    return poe_rec, elev_rec

In [ ]:
if 'isb' in params.decomp_method:
    decomp_traj_fnc = decomp_line_isb
    sphere_fnc = create_sphere_isb
    poe_elev_fnc = poe_elev_isb
elif 'phadke' in params.decomp_method:
    decomp_traj_fnc = decomp_line_phadke
    sphere_fnc = create_sphere_phadke
    poe_elev_fnc = poe_elev_phadke
else:
    raise ValueError('Only ISB and Phadke comparisons have been implemented.')

In [ ]:
start_idx = db_row['up_down_analysis'].max_run_up_start_idx
end_idx = db_row['up_down_analysis'].max_run_up_end_idx

# humerus axis is the y-axis, negate it because the y-axis points towards the humeral head center
hum_axis_rec = -db_row[params.trajectory].rot_matrix[start_idx:end_idx, :, 1]
x_traj_rec = hum_axis_rec[:, 0]
y_traj_rec = hum_axis_rec[:, 1]
z_traj_rec = hum_axis_rec[:, 2]

poe_rec, elev_rec = poe_elev_fnc()
elev_rec_deg = np.rad2deg(elev_rec)
poe_rec_deg = np.rad2deg(poe_rec)

if 'isb' in params.decomp_method:
    # print difference from start of motion for isb
    idx = np.linspace(start_idx, end_idx - 1, 11)
    elev_prints = np.rad2deg(db_row['ht'].euler.yxy_intrinsic[idx.astype(int), 1])
    poe_prints = poe_rec_deg[idx.astype(int)] - poe_rec_deg[0]
    print(', '.join('{:.2f}: {:.2f}'.format(*t) for t in zip(elev_prints, poe_prints)))

# line (as well as start and end point) for recorded humerus trajectory
hover_text = ['Elev: {:.2f}, PoE: {:.2f}'.format(elev, poe) for (elev, poe) in zip(elev_rec_deg, poe_rec_deg)]
rec_hum_line = go.Scatter3d(name='Actual Trajectory', x=x_traj_rec, y=y_traj_rec, z=z_traj_rec, mode='lines+markers',
                            line=dict(color='blue', width=2), marker=dict(size=2), hovertext=hover_text,
                            hoverinfo='text')
rec_hum_start = go.Scatter3d(name='Start', x=[x_traj_rec[0]], y=[y_traj_rec[0]], z=[z_traj_rec[0]],
                             marker=dict(size=4, color='green'), mode='markers', hoverinfo='text',
                             hovertext=[hover_text[0]])
rec_hum_end = go.Scatter3d(name='End', x=[x_traj_rec[-1]], y=[y_traj_rec[-1]], z=[z_traj_rec[-1]],
                           marker=dict(size=4, color='red'), mode='markers', hoverinfo='text', hovertext=hover_text[-1])

rec_traj = [rec_hum_start, rec_hum_end, rec_hum_line]

In [ ]:
axes = create_axes()
sphere = sphere_fnc()
decomp_traj = decomp_traj_fnc()

layout = dict(
    scene=dict(
        camera_eye=dict(x=-1.5, y=0.75, z=0.5),
        camera_up=dict(x=0, y=1, z=0),
        aspectratio=dict(x=1, y=1, z=0.5),
        xaxis=dict(showgrid=False, showbackground=False, zeroline=False, showticklabels=False, showspikes=False,
                   visible=False),
        yaxis=dict(showgrid=False, showbackground=False, zeroline=False, showticklabels=False, showspikes=False,
                   visible=False),
        zaxis=dict(showgrid=False, showbackground=False, zeroline=False, showticklabels=False, showspikes=False,
                   visible=False)),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=True,
    legend=dict(itemsizing='constant')
)
fig = go.Figure(dict(data = sphere + axes + rec_traj + [decomp_traj], layout=layout))
fig.show()